In [48]:
import os
import datetime
import re

import pandas as pd
# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

import warnings

In [49]:
movies = {'Everything Everywhere All at Once':'https://www.rottentomatoes.com/m/everything_everywhere_all_at_once/reviews',
          'CODA':'https://www.rottentomatoes.com/m/coda_2021/reviews',
          'Nomadland':'https://www.rottentomatoes.com/m/nomadland/reviews',
          'Parasite':'https://www.rottentomatoes.com/m/parasite_2019/reviews',
          'Greenbook':'https://www.rottentomatoes.com/m/green_book/reviews',
          'The Shape of Water':'https://www.rottentomatoes.com/m/the_shape_of_water_2017/reviews',
          'Moonlight':'https://www.rottentomatoes.com/m/moonlight_2016/reviews',
          'Spotlight':'https://www.rottentomatoes.com/m/spotlight_2015/reviews',
          'Birdman':'https://www.rottentomatoes.com/m/birdman_2014/reviews',
          '12 Years a Slave':'https://www.rottentomatoes.com/m/12_years_a_slave/reviews',
          'Argo':'https://www.rottentomatoes.com/m/argo_2012/reviews',
          'The Artist':'https://www.rottentomatoes.com/m/the_artist/reviews',
          'The Kings Speech':'https://www.rottentomatoes.com/m/the_kings_speech/reviews',
          'The Hurt Locker':'https://www.rottentomatoes.com/m/the_hurt_locker/reviews',
          'Slumdog Millionaire':'https://www.rottentomatoes.com/m/slumdog_millionaire/reviews'}

In [50]:
movie_data = []

for movie, url in movies.items():

    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')
        critic_reviews_section = soup.find('div', class_='layout reviews-page-container')

        if critic_reviews_section:

            critic_reviews = critic_reviews_section.find_all('div', class_="review-row")

            for review in critic_reviews:
                critic_name = review.find('a', class_='display-name').text.strip()
                critic_rating = review.find('score-icon-critic-deprecated').get('state')
                og_score = review.find('p', class_='original-score-and-url').text.strip()
                elems = og_score.split('Original Score:')
                critic_og_score = elems[1].split()[0] if len(elems) > 1 else ''
                critic_text = review.find('p', class_='review-text').text.strip()
                critic_date = review.find('span', attrs={'data-qa': 'review-date'}).text.strip()

                # create a dictionary for current movie
                movie_entry = {
                    'Movie': movie,
                    'Critic Name': critic_name,
                    'Critic Rating': critic_rating,
                    'Critic Original Score': critic_og_score,
                    'Critic Text': critic_text,
                    'Critic Date': critic_date
                }

                movie_data.append(movie_entry)

        else:
            print(f"No critic reviews found for {movie}.\n")

    else:
        print(f"Failed to retrieve data for {movie}. Status code: {response.status_code}\n")

# create dataframe with all rotten tomato reviews
rt_df = pd.DataFrame(movie_data)
rt_df

,Movie,Critic Name,Critic Rating,Critic Original Score,Critic Text,Critic Date
0,Everything Everywhere All at Once,Shahbaz Siddiqui,fresh,,Everything Everywhere All at Once is beautiful...,"Sep 27, 2023"
1,Everything Everywhere All at Once,Christian Eulinberg,fresh,,The Daniels accomplished something wonderful f...,"Sep 26, 2023"
2,Everything Everywhere All at Once,Gautam Sunder,fresh,,Michelle Yeoh finally gets a role this decade ...,"Sep 12, 2023"
3,Everything Everywhere All at Once,Sarah Vincent,fresh,3/4,The genre mashup provides a fuller insight int...,"Aug 16, 2023"
4,Everything Everywhere All at Once,Emma-Jane Betts,fresh,5/5,Everything Everywhere All at Once will remind ...,"Jul 27, 2023"
...,...,...,...,...,...,...
295,Slumdog Millionaire,Harry Kloman,rotten,,"It's a big spoonful of sugar, and I suspect it...","Feb 27, 2018"
296,Slumdog Millionaire,Virginia Montes,rotten,2/5,Slumdog Millionaire is a film loaded with clic...,"Feb 27, 2018"
297,Slumdog Millionaire,Shubhra Gupta,fresh,,It's not about poverty pornography. It's not a...,"Jan 27, 2018"
298,Slumdog Millionaire,Vadim Rizov,rotten,,"Slumdog is instantly recognizable, but rarely ...","Dec 13, 2017"
